<a href="https://www.kaggle.com/code/aliabdelmenam/used-car-price-regreesion?scriptVersionId=195741080" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/aliabdelmonam/Used-Car-Price-for-Regression/blob/main/Regreesion_for_used_car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from datetime import date
from sklearn.ensemble import IsolationForest,VotingRegressor

In [ ]:
tr= pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
ts= pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
# y_zz = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

In [ ]:
# y_zz.drop('id',axis=1,inplace=True)

In [ ]:
tr.shape

In [ ]:
tr.head()

In [ ]:
tr.info()

In [ ]:
df_train = copy.deepcopy(tr)
df_test = copy.deepcopy(ts)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# print(df_train['engine'].iloc[:150].to_string())

In [ ]:
# for col in df_train.select_dtypes(include='object'):
#     print(col)
#     print(df_train[col].nunique())
#     print(df_train[col].unique())
#     print('-----------------------------------------')

In [ ]:
# for col in df_train.select_dtypes(include='object'):
#     print(df_train[col].value_counts().sort_values(ascending=False).head(10))
#     print('----------------------------------------------------')

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.info()

# **VIS**

In [ ]:
# cols_dict = {
#       'brand':'brand',
#       'model_year':'model_year',
#       "accident": "accident",
#       "exterior color" : "ext_col",
#       "interior color" : "int_col",
# }
# target='price'

In [ ]:
# props1 = "background-color: white; color: maroon; font-weight: bold; "
# props2 = "background-color: white; color: blue; font-weight: bold; "

In [ ]:
# for label, col in cols_dict.items():
#   t=tr[[col,target]].groupby(col).agg({target: ["max", "min", "mean", "median", "size"]})
#   display(t.sort_values(by = col, ascending = True).\
#                 transpose().\
#                 style.format(formatter = '{:,.0f}').\
#                 set_caption(f"\nPrice details by {label}\n").\
#                 highlight_max(axis=1, props = props1).\
#                 highlight_min(axis=1, props = props2)
#                  )
#   print("\n\n\n")

In [ ]:
# pd.options.display.float_format = '{:.5f}'.format

In [ ]:
import re

# **Feature Extraction**

In [ ]:
df_train['Horse_power']= df_train['engine'].str.extract(r'(\d+)\.?\d*HP').astype(float)
df_train['Engine_Displacement']= df_train['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
df_train['Cylinder_Count']= df_train['engine'].str.extract(r'(\d+) Cylinder ').astype(float)

df_test['Horse_power']= df_test['engine'].str.extract(r'(\d+)\.?\d*HP').astype(float)
df_test['Engine_Displacement']= df_test['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
df_test['Cylinder_Count']= df_test['engine'].str.extract(r'(\d+) Cylinder ').astype(float)

cylinder_config={'V','Flat','Straight'}
Turbocharger = {'Turbo','Twin Turbo'}
engine_type ={'Gasoline','Electric','Hybrid'}
FUEL_SYSTEMS = {'MPFI', 'GDI', 'PDI', 'TFSI','DOHC', 'SOHC'}

In [ ]:
def zoo(word,cylinder_config):
  for i in cylinder_config:
    if i in word:
      return i
  return 'Nan'

In [ ]:
df_train['cylinder_config']=df_train['engine'].apply(zoo,args=(cylinder_config,))
df_train['Turbocharger']=df_train['engine'].apply(zoo,args=(Turbocharger,))
df_train['engine_type']=df_train['engine'].apply(zoo,args=(engine_type,))
df_train['FUEL_SYSTEMS']=df_train['engine'].apply(zoo,args=(FUEL_SYSTEMS,))

df_test['cylinder_config']=df_test['engine'].apply(zoo,args=(cylinder_config,))
df_test['Turbocharger']=df_test['engine'].apply(zoo,args=(Turbocharger,))
df_test['engine_type']=df_test['engine'].apply(zoo,args=(engine_type,))
df_test['FUEL_SYSTEMS']=df_test['engine'].apply(zoo,args=(FUEL_SYSTEMS,))

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_train['transmission'].unique()

In [ ]:
# print(df_train['transmission'].iloc[:150].to_string())

In [ ]:
# df_train['transmission'].value_counts()

In [ ]:
def extract_transmition (word):
  def get_number ():
    n=re.findall('\d+',word)
    return n[0] if n else ''
  if any(i in word for i in ['AT','A/T','At/Mt','Automatic'] ):
    return ('AT'+str(get_number()))
  elif any(i in word for i in ['CVT'] ):
     return ('CVT'+str(get_number()))
  elif any(i in word for i in ['Manual','M/T'] ):
    return ('MT'+str(get_number()))
  else :
    return 'other'

In [ ]:
df_train['new_transmission']=df_train['transmission'].apply(extract_transmition)
df_test['new_transmission']=df_test['transmission'].apply(extract_transmition)

In [ ]:
df_train[['new_transmission','transmission']].head(20)

In [ ]:
year = date.today().year
df_train['model_age'] = year - df_train['model_year']
df_test['model_age'] = year - df_test['model_year']

In [ ]:
df_train.drop(['engine','transmission','model'],axis=1,inplace=True)
df_test.drop(['engine','transmission','model'],axis=1,inplace=True)
df_train.drop('id',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)
df_train.drop(['model_year'],axis=1,inplace=True)
df_test.drop(['model_year'],axis=1,inplace=True)

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.head()

In [ ]:
y=df_train['price']
df_train=df_train.drop('price',axis=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df_train,y,test_size=.2,random_state=42)

In [ ]:
df_train.info()

# **PipeLine**

In [ ]:
num_col = df_train.select_dtypes(include='number').columns
cat_col = df_train.select_dtypes(include='object').columns

In [ ]:
num_imputer= SimpleImputer(strategy='mean')
cat_imputer= SimpleImputer(strategy='most_frequent')
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
scaler = StandardScaler()
#############################################3
for col in num_col:
  x_train[col] = num_imputer.fit_transform(x_train[[col]])
  x_test[col] = num_imputer.transform(x_test[[col]])

  df_test[col] = num_imputer.transform(df_test[[col]])

  x_train[col] = scaler.fit_transform(x_train[[col]])
  x_test[col] = scaler.transform(x_test[[col]])
  df_test[col] = scaler.transform(df_test[[col]])
##############################################
for col in cat_col:
  x_train[[col]] = cat_imputer.fit_transform(x_train[col].values.reshape(-1,1))
  x_test[[col]] = cat_imputer.transform(x_test[col].values.reshape(-1,1))

  df_test[[col]] = cat_imputer.transform(df_test[col].values.reshape(-1,1))

  x_train[col] = encoder.fit_transform(x_train[[col]])
  x_test[col] = encoder.transform(x_test[[col]])

  df_test[col] = encoder.transform(df_test[[col]])


In [ ]:
df_train.head()

In [ ]:
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
zob=pd.DataFrame(x_train,columns=df_train.columns)
zob

In [ ]:
isolation_forest = IsolationForest(contamination=0.024, random_state=42)
x_train_labels = isolation_forest.fit_predict(x_train) # 1 indicate normal, -1--> outlier (anomaly)
normal_bool = x_train_labels != -1 
x_train=x_train[normal_bool]
y_train=y_train[normal_bool]

In [ ]:
temp  = x_train.merge(y_train,left_index=True,right_index=True)
plt.figure(figsize=(15,10))
df_co=temp.corr()
mask = np.triu(np.ones_like(temp.corr())) 
sns.heatmap(df_co,annot=True,cmap='coolwarm',fmt='.3f',mask=mask)
plt.show()

In [ ]:
from scipy.stats import f_oneway
import scipy.stats as stats
# Example: Perform ANOVA test
brands = tr['brand']
prices = tr['price']
f_value, p_value = stats.f_oneway(*[prices[brands == brand] for brand in brands.unique()])
print(f"F-Value: {f_value}, P-Value: {p_value:.10f}")

In [ ]:
# x_train.drop('brand',axis=1,inplace=True)
# x_test.drop('brand',axis=1,inplace=True)
# df_test.drop('brand',axis=1,inplace=True)

In [ ]:
(1-len(x_train)/len(x_train_labels))*100

In [ ]:
import xgboost as xgb
from lightgbm import LGBMRegressor

In [ ]:
xgb=xgb.XGBRegressor(tree_method ="hist",
                          device               = "cuda" ,
                          objective             = "reg:squarederror",
                          eval_metric           = "rmse",
                          random_state          = 42,
                          colsample_bytree      = 0.45,
                          learning_rate       = 0.025,
                          max_depth          = 7,
                          n_estimators          = 3000,
                          reg_alpha          = 0.001,
                        reg_lambda            = 0.001,
                          min_child_weight      = 18,
                          verbosity = 0,
                          enable_categorical    = True,
)
lgb_params= {
        'n_epochs': 1000,
        'learning_rate': 0.01,
        'colsample_bytree': 0.55,
        'bagging_fraction': 0.8,
        'num_leaves': 3072,
        'min_child_samples': 12,
        'reg_lambda': 64, # not 128
        'max_bin': 255,
        'max_depth': 10,
        'reg_alpha': 0,
        'verbose': -1,
    }

In [ ]:
lgb_reg = LGBMRegressor(**lgb_params,loss_function='RMSE',verbosity=-1)
lgb_reg.fit(x_train,y_train)
xgb.fit(x_train,y_train)
y_pred1=xgb.predict(x_test)
y_pred= lgb_reg.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,y_pred) ))
np.sqrt(mean_squared_error(y_test,y_pred1) )

In [ ]:
vtr=VotingRegressor(estimators=[('xgboost',xgb),('lightbgm',lgb_reg)])
vtr.fit(x_train,y_train)
y_zp=vtr.predict(x_test)
mean_squared_error(y_test,y_zp,squared=False)

In [ ]:
print(r2_score(y_test,y_pred))
r2_score(y_test,y_pred1)

In [ ]:
y_p1= xgb.predict(df_test)
y_p2 = lgb_reg.predict(df_test)
y_p3 = vtr.predict(df_test)


In [ ]:
# print(np.sqrt(mean_squared_error(y_zz,y_p1) ))
# print(np.sqrt(mean_squared_error(y_zz,y_p2) ))
# print(np.sqrt(mean_squared_error(y_zz,y_p3) ))

In [ ]:
submession = pd.DataFrame({
    'id':ts['id'].values,
    'price':y_p2

})

In [ ]:
submession.head()

In [ ]:
submession.to_csv('submession.csv', index=False)